# Module 3: Scheduler

## Exercise 1: Scheduling Pods

In this exercise, you explore the Container Runtime Engines (CREs) that are installed in your Kubernetes environment.



**Objectives**

This exercise focuses on enabling you to do the following:

  - Manually schedule a pod
  - Create taints and tolerations
  - Define node affinities
  - Work with resource limits
  - Investigate static pods on the control plane
  - Deploy a scheduled pod on a control plane



## Exercise Equipment

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |



---
---

##### Task 1: Manually Schedule a Pod

In this task, you manually schedule a pod on a particular node.


In your integrated development environment (IDE), navigate to the Exercise 3 folder.


Instantiate a copy of the manual pod:


In [ ]:
kubectl create -f exercise3task1.yaml


---

Answer the following question:  

What is the state of the Pod? (Use kubectl get pods.)


In [ ]:
kubectl get pods 


---

Notice that you cannot see a pending pod in the visual Kubernetes IDE extension.


---


Answer the following question:

Why is the pod’s state pending? 

You can view the content of exercise3task1.yaml.

<details>
<summary>exercise3task1.yaml
</summary>

[exercise3task1.yaml](./exercise3task1.yaml)

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: manual
spec:
  containers:
  -  image: nginx:1.25-alpine-slim
     imagePullPolicy: IfNotPresent
     name: nginx
     resources:
      requests:
        memory: "64Mi"
        cpu: "250m"
      limits:
        memory: "128Mi"
        cpu: "500m"
  nodeName: replace-me

```

---

Update the **[exercise3task1](./exercise3task1.yaml)** file so that the **nodeName** key has a value of one of the worker node names in your environment.  
Save the file as **exercise3task1mod.yaml**



---

Instantiate a copy of the manual pod:


In [ ]:
kubectl apply -f exercise3task1mod.yaml


---

In [ ]:
kubectl get pods

Answer the following question:  

What is the state of the Pod? 



You should have succeeded in manually scheduling the pod.


Delete the pod:


In [ ]:
kubectl delete pod manual


---
---

##### Task 2: Create Taints and Tolerations

Learn how to set taints on nodes and tolerations on pods to ensure that nodes only run certain pods.


---

Answer the following question:

How many nodes are in the cluster? Use `kubectl get nodes`.


In [ ]:
kubectl get nodes

---

Answer the following question:  

Does kubwor1-1 have any taints? 

Use `kubectl describe node kubwor1-1`.


In [ ]:
kubectl describe node kubwor1-1

In [ ]:
kubectl describe node kubwor1-1|grep -i taints -b5

---

Create a taint on kubwor1-1:


In [ ]:
kubectl taint nodes kubwor1-1 app=blue:NoSchedule


---

Answer the following question:  

Does kubwor1-2 have any taints?   

Use `kubectl describe node kubwor1-2`.


In [ ]:
kubectl describe node kubwor1-2

In [ ]:
kubectl describe node kubwor1-2 |grep -i taints -b5

Create a taint on kubwor1-2:

In [ ]:
kubectl taint nodes kubwor1-2 app=blue:NoSchedule


---

Answer the following question:  

Does kubwor1-3 have any taints?

Use `kubectl describe node kubwor1-3`.


In [ ]:
kubectl describe node kubwor1-3

In [ ]:
kubectl describe node kubwor1-3|grep -i taints -b5

---

Create a taint on kubwor1-3:


In [ ]:
kubectl taint nodes kubwor1-3 app=blue:NoSchedule


---

Create a blue pod:


In [ ]:
kubectl create -f exercise3task2-blue.yaml


---

Answer the following question:  

What is the blue pod’s state, and why?  
Use `kubectl get pods -o wide`.


In [ ]:
kubectl get pods -o wide

---


Create a red pod:


In [ ]:
kubectl create -f exercise3task2-red.yaml


---

Answer the following question:  

What is the red pod’s state, and why?  

Use `kubectl get pods -o wide`.


In [ ]:
kubectl get pods -o wide

Answer the following question:  

Does the control plane have any taints?  
Use `kubectl describe node kubmas1-1`

In [ ]:
kubectl describe node kubmas1-1

In [ ]:
kubectl describe node kubmas1-1 |grep -i taints -b5

---

Remove the taint from the control plane:  


In [ ]:
kubectl taint nodes kubmas1-1 node-role.kubernetes.io/control-plane:NoSchedule-  


---

Answer the following question:  

What is the status of the red pod?


In [ ]:
kubectl get pods 

---


Answer the following question:  

What node does the red pod run on?   

Use `kubectl get pods -o wide`.



In [ ]:
kubectl get pods -o wide

---

Re-add the taint to the control plane:  


In [ ]:
kubectl taint nodes kubmas1-1 node-role.kubernetes.io/control-plane:NoSchedule


---

Answer the following question:  

What is the red pod’s status, and why?  


In [ ]:
kubectl get pods -o wide

---


Delete the red pod:


In [ ]:
kubectl delete pod red


---

Delete the blue pod:


In [ ]:
kubectl delete pod blue


---

Remove the taints on the worker nodes:


In [ ]:
kubectl taint nodes kubwor1-1 app=blue:NoSchedule-
kubectl taint nodes kubwor1-2 app=blue:NoSchedule-
kubectl taint nodes kubwor1-3 app=blue:NoSchedule-


---
---

##### Task 3: Define Node Affinities  

Create a label on a node and observe the effects of the node affinities that you set on the pods.


---

Answer the following question:  

What labels does kubwor1-1 have?  
Use `kubectl describe node kubwor1-1`.


In [ ]:
kubectl describe node kubwor1-1

---

Apply a label to the kubwor1 node:  


In [ ]:
kubectl label node kubwor1-1 app=blue


---

Create a deployment:  


In [ ]:
kubectl create deployment blue --image=nginx:1.25-alpine-slim --replicas=6


---

Answer the following question:  

Which nodes did you place lables on, and why?  

Use `kubectl get pods -o wide`.

In [ ]:
kubectl get pods -o wide

---

Delete the deployment:  


In [ ]:
kubectl delete deployment blue


---  

Configure the [exercise3task3-1.yaml](./exercise3task3-1.yaml) pod as follows:

  - Name: **blue** 
  - Replicas: **6**
  - Image: **nginx:1.25-alpine-slim**
  - NodeAffinity: **requiredDuringSchedulingIgnoredDuringExecution**
  - Key: **app**
  - Operator: **in**
  - Value: **blue**

  In dot notation:  
  
  - metadata.name=blue
  - spec.replicas=6
  - spec.template.spec.containers.1.image=nginx:1.25-alpine-slim
  - spec.template.spec.affinity.nodeAffinity.requiredDuringSchedulingIgnoredDuringExecution.nodeSelectorTerms.1.matchExpressions.1.key=app
  - spec.template.spec.affinity.nodeAffinity.requiredDuringSchedulingIgnoredDuringExecution.nodeSelectorTerms.1.matchExpressions.1.operator=In
  - spec.template.spec.affinity.nodeAffinity.requiredDuringSchedulingIgnoredDuringExecution.nodeSelectorTerms.1.matchExpressions.1.values.1=blue

Save the file as **exercise3task3-1mod.yaml**

<details>
<summary>Solution
</summary>

[Solutions/exercise3task3-1.yaml](./Solutions/exercise3task3-1.yaml)

---

Deploy the object:


In [ ]:
kubectl create -f exercise3task3-1mod.yaml


--- 

Answer the following question:

What nodes do the pods contain, and why?


In [ ]:
kubectl get pods -o wide

In [ ]:
kubectl get nodes -o wide -l app=blue

--- 

Delete the deployment:


In [ ]:
kubectl delete deployment blue


---

Configure the [exercise3task3-2.yaml](./exercise3task3-2.yaml) pod as follows:

  - Name: **red** 
  - Replicas: **6**
  - Image: **nginx:1.25-alpine-slim**
  - NodeAffinity: **requiredDuringSchedulingIgnoredDuringExecution**
  - Key: **node-role.kubernetes.io/control-plane**
  - Operator: **Exists**
  

  In dot notation:  
  
  - metadata.name=red
  - spec.replicas=6
  - spec.template.spec.containers.1.image=nginx:1.25-alpine-slim
  - spec.template.spec.affinity.nodeAffinity.requiredDuringSchedulingIgnoredDuringExecution.nodeSelectorTerms.1.matchExpressions.1.key=node-role.kubernetes.io/control-plane
  - spec.template.spec.affinity.nodeAffinity.requiredDuringSchedulingIgnoredDuringExecution.nodeSelectorTerms.1.matchExpressions.1.operator=Exists
  

Save the file as **exercise3task3-2mod.yaml**

<details>
<summary>Solution
</summary>

[Solutions/exercise3task3-2.yaml](./Solutions/exercise3task3-2.yaml)

---

Deploy the object:


In [ ]:
kubectl create -f exercise3task3-2mod.yaml


---

Answer the following question:  

What is the status of the pods, and why?


In [ ]:
kubectl get deployment

In [ ]:
kubectl get pods 

---

Remove the taint from the control plane:


In [ ]:
kubectl taint nodes kubmas1-1 node-role.kubernetes.io/control-plane:NoSchedule-


---

Answer the following questions:

What is the red pod’s status, and why?  

If the pods are running, which nodes are the pods running on, and why?


In [ ]:
kubectl get pods -o wide

--- 

Re-add the taint to the control plane:


In [ ]:
kubectl taint nodes kubmas1-1 node-role.kubernetes.io/control-plane:NoSchedule

In [ ]:
kubectl get pods

---

Answer the following question:  

What is the red pod’s status, and why?


---

Delete the deployment:  


In [ ]:
kubectl delete deployment red


---
---

##### Task 4: Work with Resource Limits  

In this task, you explore the resource limits on pods.


---

Deploy the [exercise3task4-1.yaml](./exercise3task4-1.yaml) pod.

<details>
<summary>
exercise3task4-1.yaml
</summary>

[exercise3task4-1.yaml](./exercise3task4-1.yaml)

```yaml

apiVersion: v1
kind: Pod
metadata:
  name: stress-1
spec:
  containers:
  - name: memory-demo-ctr
    image: polinux/stress:1.0.4
    imagePullPolicy: IfNotPresent
    resources:
      requests:
        memory: "100Mi"
      limits:
        memory: "200Mi"
    command: ["stress"]
    args: ["--vm", "1", "--vm-bytes", "150M", "--vm-hang", "1"]

```

In [ ]:
kubectl create -f exercise3task4-1.yaml

---

Answer the following questions:   

What is the pod’s status?  

What are the pod’s resource limits?  

How much memory does the pod require?


In [ ]:
kubectl get pods -o wide

In [ ]:
kubectl describe pod stress-1

In [ ]:
kubectl describe pod stress-1 | grep -i memory: -b10

---

Delete the stress-1 pod.


In [ ]:
kubectl delete pod stress-1

In [ ]:
kubectl get pods

---

Deploy the [exercise3task4-2.yaml](./exercise3task4-2.yaml) pod.

<details>
<summary>
exercise3task4-2.yaml
</summary>

[exercise3task4-2.yaml](./exercise3task4-2.yaml)

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: stress-2
spec:
  restartPolicy: Never
  containers:
  - name: memory-demo-ctr
    image: polinux/stress:1.0.4
    imagePullPolicy: IfNotPresent
    resources:
      requests:
        memory: "5Mi"
      limits:
        memory: "10Mi"
    command: ["stress"]
    args: ["--vm", "1", "--vm-bytes", "15M", "--vm-hang", "1"]
```

In [ ]:
kubectl create -f exercise3task4-2.yaml

In [ ]:
kubectl describe pod stress-2

---

Answer the following questions:  

What is the pod’s status?  

Notice the container report’s terminated status. 

What are the pod’s resource limits? 

How much  memory does the pod require?


---

Increase the memory limit of the stress-2 pod to 20Mi and redeploy the pod.  

(You might have to delete the existing pod and redeploy it.)

Edit [exercise3task4-2.yaml](./exercise3task4-2.yaml) and save as [exercise3task4-2mod.yaml](./exercise3task4-2mod.yaml)

In [ ]:
kubectl delete pod stress-2

In [ ]:
kubectl create -f exercise3task4-2mod.yaml

In [ ]:
kubectl describe pod stress-2

---

Answer the following question: 

What is the status of the stress-2 pod? 

It should be running.


---

Delete the stress-2 pod.


In [ ]:
kubectl delete pod stress-2

---

Deploy the exercise3task4-3.yaml pod.


Deploy the [exercise3task4-3.yaml](./exercise3task4-3.yaml) pod.

<details>
<summary>
exercise3task4-3.yaml
</summary>

[exercise3task4-3.yaml](./exercise3task4-3.yaml)

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: stress-3
spec:
  containers:
  - name: cpu-demo-ctr
    image: curtisab/stress:v1
    imagePullPolicy: IfNotPresent
    resources:
      requests:
        cpu: "0.5"
      limits:
        cpu: "0.5"
    args:
    - -cpus
    - "1"
```

In [ ]:
kubectl create -f exercise3task4-3.yaml

In [ ]:
kubectl describe pod stress-3

---

Answer the following questions: 

What is the pod’s status? 

What are the pod’s resource limits?

How much CPU does the pod require?


---

Unlike memory constraints, CPU constraints are optional. 

The application requires 1 CPU, but the app runs successfully even with 0.5 CPU.



---

Increase the CPU limit of the stress-3 pod to 2 and redeploy the pod.

(You might have to delete the existing pod and redeploy it.)

Edit [exercise3task4-3.yaml](./exercise3task4-3.yaml) and save as [exercise3task4-3mod.yaml](./exercise3task4-3mod.yaml)

In [ ]:
kubectl delete pod stress-3

In [ ]:
kubectl create -f exercise3task4-3mod.yaml

In [ ]:
kubectl describe pod stress-3

---

Answer the following question: 

What is the stress-3 pod’s status? 

It should be running.


---

Delete the stress-3 pod.


In [ ]:
kubectl delete pod stress-3

---
---

##### Task 5: Investigate Static Pods on the Control Plane

In this task, you inspect the **kudeadm** tool that installed the control-plane pods as static pods.


---

Identify the host names for all nodes in your cluster:


In [ ]:
kubectl get nodes


---

Try to identify which pods are likely static pods in your cluster:


In [ ]:
kubectl get pods --all-namespaces


---

Investigate all the control-plane pods:


In [ ]:
kubectl get pods -n kube-system -o wide


---

Answer the following question:

Are the kube-proxy pods deployed as static pods?


---


Notice that some of the control-plane pods have the kubmas1-1 hostname appended to their
pod names. 

This naming indicates that the pod is deployed as a static pod.


---

From a Secure Shell (SSH) connection to the kubmas1-1 node, locate the kubelet configuration file (look for the -config argument):


In [ ]:
ssh root@kubmas1-1 ps -ef | grep kubelet


---

Look for the staticPodPath in the kubelet configuration file:


In [ ]:
ssh root@kubmas1-1 grep -i static /var/lib/kubelet/config.yaml


---

Navigate to this location on your control plane.


---

Investigate the YAML files in the static pod path and then answer the following question:

How do the YAML files relate to the describe pod output of the corresponding pods?


In [ ]:
d=`ssh root@kubmas1-1 grep -i static /var/lib/kubelet/config.yaml| awk '{print $2}'`; ssh root@kubmas1-1 "ls -la $d"


In [ ]:
ssh root@kubmas1-1 cat "$d/etcd.yaml"


---

CHALLENGE STEP: Run a new YAML definition to run as a static pod:


In [ ]:
ssh root@kubmas1-1 "kubectl run static-box --image=busybox:1.35.0 --restart=Never --dry-run=client -o yaml -- sleep 2000 > $d/staticBox.yaml"


In [ ]:
ssh root@kubmas1-1 ls -l $d


---

CHALLENGE STEP: Verify that the new static pod is running:


In [ ]:
kubectl get pods


---

CHALLENGE STEP: Answer the following questions:

What namespaces is the new static pod running in? Why?


---

CHALLENGE STEP: Terminate the static pod:


In [ ]:
kubectl delete pods static-box-kubmas1-1


---

CHALLENGE STEP: Verify that the new static pod is not running (it restarts even though you
defined the restart option as “never”):


In [ ]:
kubectl get pods


---

CHALLENGE STEP: Delete the static pod definition:


In [ ]:
ssh root@kubmas1-1 rm $d/staticBox.yaml


In [ ]:
ssh root@kubmas1-1 ls -l $d


CHALLENGE STEP: Wait about 1 minute and answer the following question:

Is the pod running? 

Use `kubectl get pods`.


In [ ]:
kubectl get pods

---
---

##### Challenge Task 6: Deploy a Scheduled Pod on a Control Plane

In this challenge task, you deploy a pod on the control plane. 

Remember that challenge tasks and steps are optional.



---

CHALLENGE STEP: Create a pod definition file that runs on the control plane, using the
following definition:

```yaml
Pod name: controlSchedRun
Tolerations:
  - effect: NoSchedule
      key: node-role.kubernetes.io/control-plane
      operator: true
Node selector:
     node-role.kubernetes.io/control-plane: ""


---

CHALLENGE STEP: Instantiate the pod.


---

CHALLENGE STEP: Verify that the pod is running on the control plane.


In [ ]:
kubectl get pods


---

CHALLENGE STEP: Remove the pod that is running on 



In [ ]:
kubectl delete pods controlSchedRun


End of exercise



---
---